In [2]:
import urllib.request
import config
import json
import math

In [3]:
#유료 관광지 방문객 수 조회를 위한 url 생성
 
def makeURL(yyyymm, sido, gungu, nPagenum, nItems):
    end_point = "http://openapi.tour.go.kr/openapi/service/TourismResourceStatsService/getPchrgTrrsrtVisitorList"  #
    parameters = "?_type=json&serviceKey=" + config.access_key  # config.py  파일에 있는 변수. access_key="B0j5Y8uG%2BI...
    parameters += "&YM=" + yyyymm  # 요청 파라미터 참고 .
    parameters += "&SIDO=" + urllib.parse.quote(sido)  # 한글은 인코딩하여 사용
    parameters += "&GUNGU=" + urllib.parse.quote(gungu)
    parameters += "&RES_NM=&pageNo=" + str(nPagenum)  # 숫자는 문자로 변경 후 사용
    parameters += "&numOfRows=" + str(nItems)
    url = end_point + parameters
    return url

In [8]:
def requestURL(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.status == 200:                  # 200이면 정상 응답
            print("Url Request Success")
            data = response.read().decode('utf-8')
            return data                             # 크롤링 결과 반환
    except Exception as e:  # url로 요청 중 오류가 발생시 실행
        print(e)
        print("Error for URL : %s" %url)
        return None

In [9]:
def getTourPointData(item, yyyymm, jsonResult):
    addrCd = 0
    if 'addrCd'   in item :
        addrCd = item['addrCd']

    gungu = ''
    if 'gungu' in item:
        gungu = item['gungu']

    sido = ''
    if 'sido' in item:
        sido = item['sido']

    resNm = ''
    if 'resNm' in item:
        resNm = item['resNm']

    rnum = ''
    if 'rnum' in item:
        rnum = item['rnum']

    ForNum = ''
    if 'csForCnt' in item:
        ForNum = item['csForCnt']

    NatNum = ''
    if 'csNatCnt' in item:
        NatNum = item['csNatCnt']

    jsonResult.append({'yyyymm': yyyymm, 'addrCd': addrCd,
                       'gungu': gungu, 'sido': sido, 'resNm': resNm,
                       'rnum': rnum, 'ForNum': ForNum, 'NatNum': NatNum})
    return



In [7]:
def main():
    sido = '서울특별시'
    gungu = ''
    nPagenum = 1
    nItems = 100
    nStartYear = 2011
    nEndYear = 2018
    jsonResult = []

    for year in range(nStartYear, nEndYear):    #시작년도부터 종료년도(endyear-1) 까지 반복 요청
        for month in range(1, 13):              #1월~12월까지 반복

            # 201101, 201102, 201103, 201104 .... 을 하나씩 생성
            yyyymm =str(year)
            if month < 10 :
                yyyymm = yyyymm+'0' +str(month)
            else :
                yyyymm = yyyymm +  str(month)

            nPagenum = 1

            while True:
                targetURL = makeURL(yyyymm, sido, gungu, nPagenum, nItems)  # 1. URL 만들기
                result = requestURL(targetURL)                              # 2. URL로 크롤링
                #print(result)                                               # 3. 크롤링 결과 출력

                if (result != None):
                    jsonData = json.loads(result)

                if (jsonData['response']['header']['resultMsg'] == 'OK'):  # 요청에 대한 응답이 정상이면
                    nTotal = jsonData['response']['body']['totalCount']  # 요청 결과 건수

                    if nTotal == 0:  # 해당 페이지에 데이터가 없는 경우 반복 나옴.
                        break

                    for item in jsonData['response']['body']['items']['item']:  # 응답 결과 건수 만큼 반복 돌면서 data 저장
                        getTourPointData(item, yyyymm, jsonResult)

                    nPage = math.ceil(nTotal / 100)  # 이 사이트는 한번에 100개의 데이터를 요청.
                    if (nPagenum == nPage):          # 전체 데이터가 100 이상인 경우 총 페이지 수를 계산하여 여러번 반복해야 함.
                        break

                    nPagenum += 1                    # nPagenum를 증가 시켜가면 계속 요청
                else:
                    break


    fileName ='%s_관광지입장정보_%d_%d.json' % (sido, nStartYear, nEndYear - 1)

    with open(fileName, 'w',
              encoding='utf8') as outfile:      # 파일을 저장하기위해 파일 오픈(파일 없는 경우 생성된다)
        retJson = json.dumps(jsonResult,         # jsonResult(딕셔너리, 리스트)를 JSON 형태의 문자열로 생성
                             indent=4 ,          # indent를 주면 들여쓰기
                             ensure_ascii=False) # 한글 저장 시 ensure_ascii=False
        outfile.write(retJson)  # 파일에 저장

    print(fileName ," 저장 성공")

main()

Url Request Success
{"response":{"header":{"resultCode":"0000","resultMsg":"OK"},"body":{"items":{"item":[{"addrCd":1111,"csForCnt":14137,"csNatCnt":43677,"gungu":"종로구","resNm":"창덕궁","rnum":1,"sido":"서울특별시","ym":201101},{"addrCd":1111,"csNatCnt":6523,"gungu":"종로구","resNm":"운현궁","rnum":2,"sido":"서울특별시","ym":201101},{"addrCd":1111,"csForCnt":40224,"csNatCnt":141183,"gungu":"종로구","resNm":"경복궁","rnum":3,"sido":"서울특별시","ym":201101},{"addrCd":1111,"csForCnt":697,"csNatCnt":17644,"gungu":"종로구","resNm":"창경궁","rnum":4,"sido":"서울특별시","ym":201101},{"addrCd":1111,"csForCnt":6837,"csNatCnt":11486,"gungu":"종로구","resNm":"종묘","rnum":5,"sido":"서울특별시","ym":201101},{"addrCd":1117,"csNatCnt":260272,"gungu":"용산구","resNm":"국립중앙박물관","rnum":6,"sido":"서울특별시","ym":201101},{"addrCd":1111,"csNatCnt":57787,"gungu":"종로구","resNm":"서울역사박물관","rnum":7,"sido":"서울특별시","ym":201101},{"addrCd":1114,"csForCnt":5743,"csNatCnt":92135,"gungu":"중구","resNm":"덕수궁","rnum":8,"sido":"서울특별시","ym":201101},{"addrCd":1114,"csNatCnt":2345